# Importar Lib's

In [1]:
import os
import openpyxl
import pandas as pd
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from datetime import datetime
from ipywidgets import interact
from openpyxl import Workbook

# Passo 1: Listar arquivos PDF em uma pasta específica

In [2]:
diretorio_lote = r"C:\Users\helym\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\AC\1.0-CORRETIVA\2ªTRANCHE"
#"C:\Users\HELY-DELL\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\1.0-CORRETIVA\2ªTRANCHE"

In [3]:
df_filtered_global_oem = None

def ler_planilha():
    """Lê uma planilha do Excel."""
    excluir_status = ['TREINAMENTO','DUPLICADO','CANCELADO ']
    df_bd = pd.read_excel(r"C:\Users\helym\projetos_python\jupyter\OEM\content\base_sip_Concluido.xlsx")
    df = df_bd[~df_bd['STATUS'].isin(excluir_status)]
    return df

def aplicar_filtros(rota_value, status_value, tipo_value, executor_value, origem_value, tipocausa_value, data_inicial, data_final):
    """Aplica filtros a uma planilha."""
    global df_filtered_global_oem
    df_filtered_global_oem = df[
        df["ROTA"].isin(rota_value) &
        df["STATUS"].isin(status_value) &
        df["TIPO"].isin(tipo_value) &
        df["EXECUTOR"].isin(executor_value) &
        df["ORIGEM"].isin(origem_value) &
        df["TIPOCAUSA"].isin(tipocausa_value) &
        (df["DATACONCLUSAO"] >= data_inicial) &
        (df["DATACONCLUSAO"] <= data_final)
    ]
    return df_filtered_global_oem

if __name__ == "__main__":
    df = ler_planilha()

    df["DATACONCLUSAO"] = pd.to_datetime(df["DATACONCLUSAO"]).dt.date

    rota_options = sorted(list(df['ROTA'].unique()))
    rota_filter = widgets.SelectMultiple(options=rota_options, description="Rotas:")

    status_options = sorted(list(df['STATUS'].unique()))
    status_filter = widgets.SelectMultiple(options=status_options, description="Status:")

    tipo_options = sorted(list(df['TIPO'].unique()))
    tipo_filter = widgets.SelectMultiple(options=tipo_options, description="Tipo:")

    executor_options = sorted(list(df['EXECUTOR'].unique()))
    executor_filter = widgets.SelectMultiple(options=executor_options, description="Usuário:")

    origem_options = sorted(list(df['ORIGEM'].unique()))
    origem_filter = widgets.SelectMultiple(options=origem_options, description="Origem:")

    tipocausa_options = sorted(list(df['TIPOCAUSA'].unique()))
    tipocausa_filter = widgets.SelectMultiple(options=tipocausa_options,descrition="Tipo Causa")

    data_inicial_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].min())
    data_final_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].max())

    aplicar_filter_button = widgets.Button(description="Aplicar filtros")

    def on_button_click(event):
        global df_filtered_global_oem
        df_filtrado = aplicar_filtros(
            rota_filter.value, status_filter.value, tipo_filter.value,
            executor_filter.value, origem_filter.value,
            tipocausa_filter.value,
            data_inicial_filter.value, data_final_filter.value
        )
        df_filtered_global_oem = df_filtrado

    aplicar_filter_button.on_click(on_button_click)

    display(data_inicial_filter)
    display(data_final_filter)
    df_filtered_global_oem
    display(
        widgets.HBox([origem_filter, tipo_filter, rota_filter, executor_filter, status_filter],
                     layout=widgets.Layout(align_items='baseline'))
    )
    display(
        widgets.HBox([tipocausa_filter, aplicar_filter_button],
                     layout=widgets.Layout(align_items='center'))
    )

DatePicker(value=datetime.date(2022, 1, 1), step=1)

DatePicker(value=datetime.date(2024, 5, 9), step=1)

## Pasta contendo os arquivos PDF

In [4]:
def encontrar_duplicados(numocorrencia):
    duplicados = numocorrencia[numocorrencia.duplicated(subset='NUMOCORRENCIA')]
    return duplicados

arquivos_pdf = []
for raiz, diretorios, arquivos in os.walk(diretorio_lote):
    for arquivo in arquivos:
        if arquivo.endswith('.pdf'):
            arquivos_pdf.append(os.path.join(raiz, arquivo))

string = pd.DataFrame(pd.Series(arquivos_pdf).str.extract(r"(\d+)_(\d+)_(.+)_(.+)_(.+)_(.+)"))
string.rename(columns={1: 'NUMOCORRENCIA'}, inplace=True)
numocorrence = string[['NUMOCORRENCIA']].astype('float')  # Convertendo para float para acomodar NaN

# Contagem total de duplicatas
duplicadas_count = numocorrence[numocorrence.duplicated(subset='NUMOCORRENCIA')].shape[0]

# Encontrar duplicatas
date_duplicated= encontrar_duplicados(numocorrence.copy())  # Cria uma cópia independente
date_duplicated.dropna(inplace=True)  # Remove NaNs do DataFrame duplicatas
date_duplicated['NUMOCORRENCIA'] = date_duplicated['NUMOCORRENCIA'].astype('int')  # Convertendo de volta para int
# Mesclando os dados principais.
df_pdf = numocorrence.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left' )
df_pdf = df_pdf[['NUMOS', 'UC','IDSIGFI','NUMOCORRENCIA','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]
print("Número total de duplicatas:", duplicadas_count)
print("Duplicatas encontradas após remoção de NaN:")
display(df_pdf)

Número total de duplicatas: 1
Duplicatas encontradas após remoção de NaN:


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
0,202400004455,651853.0,41792,12014.0,GRACIENE DA SILVA SOUZA,22.0,VALIDADO,ONEOM007,2024-02-04 16:13:00
1,202400005544,713951.0,18707,12570.0,MARCELO MELO DE CARVALHO,16.0,VALIDADO,ONEL004,2024-02-01 09:30:00
2,202400009199,714455.0,16842,18821.0,ELIVANEI DA SILVA MONTEIRO,14.0,VALIDADO,ONEOM005,2024-02-18 10:12:00
3,202400006914,714915.0,19723,13497.0,MARIA JERCINA RODRIGUES DO NASCIMENTO,21.0,VALIDADO,ONEOM007,2024-02-05 14:25:00
4,202400006922,715187.0,19623,13890.0,OZELINA LIMA MONTEIRO,21.0,VALIDADO,ONEOM007,2024-02-06 10:47:00
...,...,...,...,...,...,...,...,...,...
96,202400001170,NaN,41322,NaN,MARIA ANTÔNIA DE SOUZA GOMES,21.0,VALIDADO,ONEOM014,2024-01-07 15:05:00
97,202400000495,NaN,21134,NaN,RAIMUNDO NONATO CONCEIÇÃO CAMPOS,21.0,VALIDADO,ONEOM014,2024-01-06 13:03:00
98,202400000487,NaN,41542,NaN,NATANAEL FERREIRA FEITOSA,21.0,VALIDADO,ONEOM014,2024-01-08 10:37:00
99,202400000305,NaN,20073,NaN,MARIA FRANCISCA MOURA MESQUITA,21.0,VALIDADO,ONEOM014,2024-01-05 15:47:00


# Lista de arquivos PDF na pasta

# Passo 2: Ler o arquivo Excel com a lista de arquivos

In [5]:
caminho_arquivo_excel = df_filtered_global_oem
caminho_arquivo_excel['NUMOCORRENCIA'].count()
df_excel = caminho_arquivo_excel [['NUMOS', 'UC','IDSIGFI','NUMOCORRENCIA','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]

In [6]:
df_excel

,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
52,202400029205,NaN,42174,65920,FRANCISCO EMERSON BEZERRA RODRIGUES,21.0,VALIDADO,ONEOM014,2024-04-26 15:53:00
53,202400029197,NaN,19473,65479,MARIA SOCORRO CORDOVEZ DE PAIVA,21.0,VALIDADO,ONEOM014,2024-04-27 08:28:00
94,202400028330,NaN,35454,61557,PEDRO ANTÔNIO FRANÇA,21.0,VALIDADO,ONEOM014,2024-04-27 10:55:00
95,202400028322,NaN,35462,61436,JOÃO BATISTA FRANÇA,21.0,VALIDADO,ONEOM006,2024-04-27 09:24:00
96,202400028314,NaN,19469,61482,GRASSONILDO PAIVA DA SILVA,21.0,VALIDADO,ONEOM006,2024-04-26 10:36:00
...,...,...,...,...,...,...,...,...,...
2680,202300038900,714171.0,16729,163433,JOSE DEUSIMAR PEREIRA DOS SANTOS,16.0,VALIDADO,ONEOM002,2023-12-15 15:13:00
2710,202300038595,714268.0,16769,151641,RAIMUNDO NONATO ALMEIDA DE OLIVEIRA,16.0,VALIDADO,ONEOM002,2023-12-15 13:50:00
2711,202300038587,713900.0,16719,153092,FRANCISCO LIMA GOMES,16.0,VALIDADO,ONEOM002,2023-12-21 10:26:00
2728,202300038405,716719.0,19603,159626,FRANCISCO GERLON DE ARAÚJO FERREIRA,21.0,VALIDADO,ONEOM014,2023-12-19 15:10:00


## DataFrame a partir do arquivo Excel


#  Passo 3: Comparar os DataFrames para identificar arquivos ausentes no PDF e arquivos não listados no Excel

In [7]:
arquivos_ausentes_no_pdf = df_excel[~df_excel['NUMOCORRENCIA'].isin(df_pdf['NUMOCORRENCIA'])]
arquivos_nao_listados_no_excel = df_pdf[~df_pdf['NUMOCORRENCIA'].isin(df_excel['NUMOCORRENCIA'])]

## Resultados

In [8]:
print(arquivos_ausentes_no_pdf['NUMOCORRENCIA'].count())
display(arquivos_ausentes_no_pdf)

56


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
52,202400029205,NaN,42174,65920,FRANCISCO EMERSON BEZERRA RODRIGUES,21.0,VALIDADO,ONEOM014,2024-04-26 15:53:00
53,202400029197,NaN,19473,65479,MARIA SOCORRO CORDOVEZ DE PAIVA,21.0,VALIDADO,ONEOM014,2024-04-27 08:28:00
94,202400028330,NaN,35454,61557,PEDRO ANTÔNIO FRANÇA,21.0,VALIDADO,ONEOM014,2024-04-27 10:55:00
95,202400028322,NaN,35462,61436,JOÃO BATISTA FRANÇA,21.0,VALIDADO,ONEOM006,2024-04-27 09:24:00
96,202400028314,NaN,19469,61482,GRASSONILDO PAIVA DA SILVA,21.0,VALIDADO,ONEOM006,2024-04-26 10:36:00
97,202400028306,NaN,42192,60979,ELIAS LEAL SILVA,21.0,VALIDADO,ONEOM014,2024-04-26 13:19:00
98,202400028280,NaN,20149,58932,JOSÉ JOSIANO DE SOUZA DO CARMO,21.0,VALIDADO,ONEOM006,2024-04-26 09:16:00
99,202400028272,NaN,42542,55433,ILTON DE MENEZES FERREIRA,21.0,VALIDADO,ONEOM014,2024-04-25 12:12:00
100,202400028264,NaN,20077,54554,VIVIANE DA SILVA DUARTE,21.0,VALIDADO,ONEOM014,2024-04-25 10:29:00
130,202400027373,17553.0,17553,63324,CLESSIANE PEREIRA,17.0,VALIDADO,ONEOM013,2024-04-23 12:41:00


In [9]:
arquivos_nao_listados_no_excel['NUMOCORRENCIA'].count()
nao_listado_excel = arquivos_nao_listados_no_excel.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left' )
nao_listado_excel

,NUMOS_x,UC_x,IDSIGFI_x,NUMOCORRENCIA,NOMECLIENTE_x,ROTA_x,STATUS_x,EXECUTOR_x,DTCONCLUSAO_x,Unnamed: 0,...,IDDIAGNOSTICO,DTINIDESLOCAMENTO,DTCHEGADA,DTAHORAFATO,STATUSSRV,DATACONCLUSAO,IDMOBILE,MANUTENCAO,WHMAX,DATALOG


In [10]:
recorrence= df_filtered_global_oem.merge(date_duplicated, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left')
recorrence['IDSIGFI'] = recorrence['IDSIGFI'].astype('int')

In [11]:
duplicated_numocorrencia = date_duplicated.merge(df_filtered_global_oem, left_on='NUMOCORRENCIA', right_on='NUMOCORRENCIA', how='left')

In [12]:
print("listar arquivo duplicados:")
print(recorrence.shape)
display(recorrence)

print("listar nº ocorrencia que esta duplicada")
display(sorted(duplicated_numocorrencia['NUMOCORRENCIA']))  

print("Arquivos ausentes na lista de PDF:")
print(arquivos_ausentes_no_pdf.shape)
display(arquivos_ausentes_no_pdf)

print("\nArquivos não listados no Excel:")
print(arquivos_nao_listados_no_excel.shape)
display(arquivos_nao_listados_no_excel)

listar arquivo duplicados:
(126, 52)


,Unnamed: 0,ORIGEM,TIPO,NUMOS,NUMOCORRENCIA,ROTA,NOMECLIENTE,CPF,CODIGOENERGISA,UC,...,IDDIAGNOSTICO,DTINIDESLOCAMENTO,DTCHEGADA,DTAHORAFATO,STATUSSRV,DATACONCLUSAO,IDMOBILE,MANUTENCAO,WHMAX,DATALOG
0,False,ENERGISA,CORRETIVA,202400029205,65920,21.0,FRANCISCO EMERSON BEZERRA RODRIGUES,082.722.902-00,1,NaN,...,34,2024-04-26 13:56:00,2024-04-26 13:56:00,2024-04-25 15:44:00,3,2024-04-26,202400029205,Report2,1500,NaN
1,False,ENERGISA,CORRETIVA,202400029197,65479,21.0,MARIA SOCORRO CORDOVEZ DE PAIVA,002428822-50,002428822-50,NaN,...,23,2024-04-27 07:22:00,2024-04-27 07:22:00,2024-04-25 15:42:00,3,2024-04-27,202400029197,Report2,2666,NaN
2,False,ENERGISA,CORRETIVA,202400028330,61557,21.0,PEDRO ANTÔNIO FRANÇA,699.837.982-04,1,NaN,...,23,2024-04-27 08:53:00,2024-04-27 08:53:00,2024-04-24 11:25:00,3,2024-04-27,202400028330,Report2,2666,NaN
3,False,ENERGISA,CORRETIVA,202400028322,61436,21.0,JOÃO BATISTA FRANÇA,72300248204,1,NaN,...,23,2024-04-27 08:51:00,2024-04-27 08:51:00,2024-04-24 11:22:00,3,2024-04-27,202400028322,Report2,2666,NaN
4,False,ENERGISA,CORRETIVA,202400028314,61482,21.0,GRASSONILDO PAIVA DA SILVA,061926942-14,061926942-14,NaN,...,23,2024-04-26 09:51:00,2024-04-26 10:36:00,2024-04-24 11:18:00,3,2024-04-26,202400028314,Report2,2666,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,False,ENERGISA,CORRETIVA,202300038900,163433,16.0,JOSE DEUSIMAR PEREIRA DOS SANTOS,71183975201,L1.2023-0862,714171.0,...,23,2023-12-15 13:54:00,2023-12-15 14:35:00,2023-12-13 18:08:00,3,2023-12-15,202300038900,Report2,2666,NaN
122,False,ENERGISA,CORRETIVA,202300038595,151641,16.0,RAIMUNDO NONATO ALMEIDA DE OLIVEIRA,79203345272,L1.2023-1207,714268.0,...,23,2023-12-15 12:21:00,2023-12-15 13:50:00,2023-12-12 15:42:00,3,2023-12-15,202300038595,Report2,2666,NaN
123,False,ENERGISA,CORRETIVA,202300038587,153092,16.0,FRANCISCO LIMA GOMES,88089495249,L1.2023-0766,713900.0,...,23,2023-12-21 09:28:00,2023-12-21 09:54:00,2023-12-12 15:39:00,3,2023-12-21,202300038587,Report2,2666,NaN
124,False,ENERGISA,CORRETIVA,202300038405,159626,21.0,FRANCISCO GERLON DE ARAÚJO FERREIRA,031025272-56,031025272-56,716719.0,...,23,2023-12-19 12:23:00,2023-12-19 12:23:00,2023-12-08 17:24:00,3,2023-12-19,202300038405,Report2,2666,NaN


listar nº ocorrencia que esta duplicada


[]

Arquivos ausentes na lista de PDF:
(56, 9)


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
52,202400029205,NaN,42174,65920,FRANCISCO EMERSON BEZERRA RODRIGUES,21.0,VALIDADO,ONEOM014,2024-04-26 15:53:00
53,202400029197,NaN,19473,65479,MARIA SOCORRO CORDOVEZ DE PAIVA,21.0,VALIDADO,ONEOM014,2024-04-27 08:28:00
94,202400028330,NaN,35454,61557,PEDRO ANTÔNIO FRANÇA,21.0,VALIDADO,ONEOM014,2024-04-27 10:55:00
95,202400028322,NaN,35462,61436,JOÃO BATISTA FRANÇA,21.0,VALIDADO,ONEOM006,2024-04-27 09:24:00
96,202400028314,NaN,19469,61482,GRASSONILDO PAIVA DA SILVA,21.0,VALIDADO,ONEOM006,2024-04-26 10:36:00
97,202400028306,NaN,42192,60979,ELIAS LEAL SILVA,21.0,VALIDADO,ONEOM014,2024-04-26 13:19:00
98,202400028280,NaN,20149,58932,JOSÉ JOSIANO DE SOUZA DO CARMO,21.0,VALIDADO,ONEOM006,2024-04-26 09:16:00
99,202400028272,NaN,42542,55433,ILTON DE MENEZES FERREIRA,21.0,VALIDADO,ONEOM014,2024-04-25 12:12:00
100,202400028264,NaN,20077,54554,VIVIANE DA SILVA DUARTE,21.0,VALIDADO,ONEOM014,2024-04-25 10:29:00
130,202400027373,17553.0,17553,63324,CLESSIANE PEREIRA,17.0,VALIDADO,ONEOM013,2024-04-23 12:41:00



Arquivos não listados no Excel:
(0, 9)


,NUMOS,UC,IDSIGFI,NUMOCORRENCIA,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO


In [14]:
#Salvando a planilha Manutenções Energisa_preventiva para ser enviadas em Excell( .xlsx)
dataframe_excel = pd.ExcelWriter(r"C:\Users\helym\projetos_python\jupyter\OEM\output\pdf_ausentes_corretivas_2_TRANCHE-geral.xlsx")
arquivos_ausentes_no_pdf.to_excel(dataframe_excel)
dataframe_excel.close()